In [ ]:
import pandas as pd
import numpy as np
import os
import time
import datetime
import openai
import pymssql
import re
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from threading import Thread
import functools
from tableone import TableOne
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu as mwu
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from sksurv.nonparametric import kaplan_meier_estimator
from sklearn.metrics import roc_curve, auc
import random

table_name="" #enter table name
df=pd.read_csv(table_name+'/'+table_name+'.csv')
seednum=3
random.seed(seednum)
rrr=list(range(len(df)))
random.shuffle(rrr)

conn = pymssql.connect()# enter connection info

openai.api_key = "" #enter api key

df=df.iloc[rrr]
df.reset_index(inplace=True,drop=True)


input_text_temp=df['input_text5']


iterations=5
upto=50000


usercontent_global1='''Estimate the risk (in percentages) of developing a cardiovascular disease within 10 years for the person below.

'''

usercontent_global2='''

Please answer exactly in the format below, without blank lines, and no further information or answer is required.
Risk percentage=(in percentages, round to one decimal place)'''


############################


eid=df['eid']
Framingham=df['Framingham']
ACC_AHA=df['ACC/AHA']

from threading import Thread
import functools

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco


@timeout(0.37)
def ChatGPT_main(i,texttype,temper):
    input_text=texttype
    if input_text[i]!='':
        try:
            usercontent=usercontent_global1
            usercontent+=input_text[i]
            usercontent+=usercontent_global2

            messages=[
                    {"role": "user", "content": usercontent}
                ]

            completion=openai.ChatCompletion.create(model="gpt-4", 
                                                    messages=messages,
                                                    temperature=temper)


            aa=completion.choices[0].message.content.split('\n')
            #print(aa)
            remove_set={''}
            aaa=[i for i in aa if i not in remove_set]

            temp1='blank'
            temp2=aaa[0].split('=')[1]
            temp3='blank'
            temp4=Framingham[i]
            temp5=ACC_AHA[i]

        except:
            temp1=np.nan
            temp2=np.nan
            temp3=np.nan
            temp4=np.nan
            temp5=np.nan
            except_else_num+=1
    else:
        temp1=np.nan
        temp2=np.nan
        temp3=np.nan
        temp4=np.nan
        temp5=np.nan
        except_else_num+=1

    conn = pymssql.connect()# enter connection info

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql, (str(eid[i]),temp1,temp2,temp3,temp4,temp5))
            conn.commit()


In [ ]:
############################
table_name="" #enter table name
############################
for temper in [0.4]:
    for i in range(5,5+iterations):   
        newtablaname=table_name + '_' + str(int(temper*10)) + '_' + str(i)
        sql_createtable="CREATE TABLE [" + newtablaname +"""] 
        (
            eid    NVARCHAR(20),
            system    NVARCHAR(max),
            score      NVARCHAR(max),
            category     NVARCHAR(max) ,
            framingham  NVARCHAR(20),
            ACC_AHA  NVARCHAR(20)
        )

        """
        conn = pymssql.connect()# enter connection info
        with conn:
            with conn.cursor() as cur:
                cur.execute(sql_createtable)
                conn.commit()
        time.sleep(1)        
        sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"

        except_else_num=0
        time1=datetime.datetime.now()
        for j in range(upto):
            while True:
                if j%1000==0:
                    print(j)
                ChatGPT_main(j,input_text_temp,temper)
                break

        time.sleep(5)


        time2=datetime.datetime.now()
        print(time2-time1)
        print('except_else_num = ',except_else_num)

In [ ]:
# check & fill missing rows
for temper in [0.4]:
    while True:
        toggle=0
        for i in range(5,5+iterations):   
            newtablaname=table_name + '_' + str(int(temper*10)) + '_' + str(i)
            
            try:
                sql_createtable="CREATE TABLE [" + newtablaname +"""] 
                (
                    eid    NVARCHAR(20),
                    system    NVARCHAR(max),
                    score      NVARCHAR(max),
                    category     NVARCHAR(max) ,
                    framingham  NVARCHAR(20),
                    ACC_AHA  NVARCHAR(20)
                )

                """
                conn = pymssql.connect()# enter connection info
                with conn:
                    with conn.cursor() as cur:
                        cur.execute(sql_createtable)
                        conn.commit()
                time.sleep(1)
            except:
                pass
            

            sql_statement="select * from ["+ newtablaname + "]"
            conn = pymssql.connect()# enter connection info
            globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)
            globals()['data{}'.format(i)]=globals()['data{}'.format(i)].astype({'eid':int})
            globals()['data{}_eid'.format(i)]=list(globals()['data{}'.format(i)]['eid'])
            countaaaaa=0
            countcount=0
            time1=datetime.datetime.now()
            for j in range(upto):
                if df['eid'][j] not in globals()['data{}_eid'.format(i)]:

                    sql = "INSERT INTO [" + newtablaname+ "] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                    ChatGPT_main(j,input_text_temp,temper)
                    countcount+=1
                    if countcount%200==1:
                        print(j)
                        time2=datetime.datetime.now()
                        print(time2-time1)
                    countaaaaa+=1

            time.sleep(10)
            if countaaaaa==0:
                toggle=1
        if toggle==1:
            break
    print('====temper',temper,'done=======')

In [ ]:
############################
table_name="" #enter table name
iterations=5
upto=50000
############################

for numnumnumnum in range(1):
    for temper in [0.4]:
    #for temper in [1]:

        txt=''

        cutofftxt=''
        for i in range(iterations):
            newtablename=table_name + '_' + str(int(temper*10)) + '_' + str(i+numnumnumnum)

            sql_statement="select * from ["+ newtablename + "]"
            conn = pymssql.connect()# enter connection info
            globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)
            globals()['data{}'.format(i)]['rank_by_eid']=globals()['data{}'.format(i)].groupby('eid')['system'].rank(method='first')
            globals()['data{}'.format(i)]=globals()['data{}'.format(i)][globals()['data{}'.format(i)]['rank_by_eid']==1]
            globals()['data{}'.format(i)].reset_index(inplace=True,drop=True)
            globals()['data{}'.format(i)].drop(['rank_by_eid'],axis=1,inplace=True)


        for j in range(iterations):
            dat=globals()['data{}'.format(j)]

            eid=dat['eid']
            score_gpt=dat['score']
            category=dat['category']
            score_framingham=dat['framingham']
            score_acc_aha=dat['ACC_AHA']

            score_gpt2=[]

            for i in range(len(score_gpt)):
                try:
                    score_gpt2.append(float(re.findall("\d+[.]\d+[%]",score_gpt[i])[0].split('%')[0]))
                except:
                    try:
                        score_gpt2.append(float(re.findall("\d+[%]",score_gpt[i])[0].split('%')[0]))
                    except:
                        try:
                            score_gpt2.append(float(re.findall("\d+[.]\d+",score_gpt[i])[0]))
                        except:
                            try:
                                score_gpt2.append(float(re.findall("\d+",score_gpt[i])[-1]))
                            except:
                                #score_gpt2.append(np.nan)
                                score_gpt2.append(float(score_framingham[i]))
                                print(score_gpt[i],'====')


            category2=[]
            for i in range(len(score_gpt2)):
                try:
                    if np.isnan(score_gpt2[i])==False:
                        if score_gpt2[i]>20:
                            category2.append('High')
                        elif score_gpt2[i]>10:
                            category2.append('Moderate')
                        else:
                            category2.append('Low')
                    else:
                        category2.append(np.nan)
                except:
                    category2.append('Low')






            columnname_score='score'+str(j)
            columnname_cat='category'+str(j)
            globals()['data{}_1'.format(j)]=pd.DataFrame({'eid':eid,columnname_score:score_gpt2,columnname_cat:category2})
            globals()['data{}_1'.format(j)]=globals()['data{}_1'.format(j)].dropna(axis=0)
            globals()['data{}_1'.format(j)].reset_index(inplace=True,drop=True)

        df2_0=df.iloc[:upto]

        for i in range(iterations):
            globals()['data{}_1'.format(i)] = globals()['data{}_1'.format(i)].astype({ 'eid' : 'int' })

        for i in range(iterations):
            globals()['df2_{}'.format(i+1)] = pd.merge(globals()['df2_{}'.format(i)], globals()['data{}_1'.format(i)], left_on='eid', right_on='eid', how='left')

        globals()['df2_{}'.format(1)].drop_duplicates(inplace=True,ignore_index=True)
        print("length",len(globals()['df2_{}'.format(1)]))

        for j in range(iterations):
            globals()['category{}_int'.format(j)]=[]

        for j in range(iterations):
            columnname='category'+str(j)
            for i in range(len(globals()['df2_{}'.format(iterations)])):
                if globals()['df2_{}'.format(iterations)][columnname][i]=='Low':
                    globals()['category{}_int'.format(j)].append(1)
                elif globals()['df2_{}'.format(iterations)][columnname][i]=='Moderate':
                    globals()['category{}_int'.format(j)].append(2)
                else:
                    globals()['category{}_int'.format(j)].append(3)

        df_final=globals()['df2_{}'.format(iterations)]

        for j in range(iterations):
            columnname='category'+str(j)+'_int'
            df_final[columnname]=globals()['category{}_int'.format(j)]

        score_final=[]
        category_final=[]
        for i in range(len(df_final)):
            score_temp=[]
            category_temp=[]
            for j in range(iterations):
                columnname1='score'+str(j)
                columname2='category'+str(j)+'_int'
                score_temp.append(df_final[columnname1][i])
                category_temp.append(df_final[columname2][i])

            score_temptemp=np.nanmean(score_temp)
            category_temptemp=np.nanmedian(category_temp)
            score_final.append(score_temptemp)
            category_final.append(category_temptemp)

        df_final['score_final']=score_final
        df_final['category_final']=category_final


        scorefinallist=list(df_final['score_final'])
        t1=np.percentile(scorefinallist,100/3)
        t2=np.percentile(scorefinallist,200/3)
        newcatfinal=[]
        for i in range(len(df_final)):
            if df_final['score_final'][i]<=t1:
                newcatfinal.append(1)
            elif df_final['score_final'][i]<=t2:
                newcatfinal.append(2)
            else:
                newcatfinal.append(3)
        df_final['category_final']=newcatfinal



        cig_final=[]
        for i in range(len(df_final)):
            if df_final['smoking'][i]==2:
                cig_final.append(1)
            else:
                cig_final.append(0)
        df_final['Smoking']=cig_final

        df_final['death_date']=pd.to_datetime(df_final['death_date'])
        df_final['assess_date']=pd.to_datetime(df_final['assess_date'])
        df_final['datediff']=df_final['death_date']-df_final['assess_date']

        datediff_int=[]
        for i in range(len(df_final)):
            datediff_int.append(df_final['datediff'][i].days)

        df_final['datediff_int']=datediff_int

        datediff_int2=[]
        event=[]
        for i in range(len(df_final)):
            if np.isnan(df_final['datediff_int'][i])==True:
                datediff_int2.append(3650)
                event.append(False)
            else:
                if df_final['datediff_int'][i]>3650:
                    datediff_int2.append(3650)
                    event.append(False)
                else:
                    datediff_int2.append(df_final['datediff_int'][i])
                    event.append(True)
        df_final['datediff_int2']=datediff_int2
        df_final['event']=event

        aa=list(df_final['datediff_int2'])
        aa2=[x for x in aa if x < 3650]
        aaa=[x for x in aa if x >=0]
        aaaa=[x for x in aaa if x < 3650]
        exclusion_event_before_assessment=len(aa2)-len(aaaa)
        bb=exclusion_event_before_assessment

        print('allcausemortality_within10years=',len(aa2)-bb,'/',upto-bb, ", exclusion_event_before_assessment=",bb)

        txt+= (  'allcausemortality_within10years=' + str(len(aa2)-bb) + ' / ' + str(upto-bb)+ " , exclusion_event_before_assessment= "+ str(bb) + '\n'  )

        algo=pd.read_csv('ukb_algorithmic_outcome.csv')
        algo=algo[['eid','Min_Date']]
        algo.columns=['eid','algo_date']
        algo['algo_date']=pd.to_datetime(algo['algo_date'])

        firstoccur=pd.read_csv('ukb_new_add_first_occur.csv')
        firstoccur=firstoccur[['eid','Min_Date']]
        firstoccur.columns=['eid','firstoccur_date']
        firstoccur['firstoccur_date']=pd.to_datetime(firstoccur['firstoccur_date'])

        df_final=pd.merge(df_final, algo, left_on='eid', right_on='eid', how='left')
        df_final=pd.merge(df_final, firstoccur, left_on='eid', right_on='eid', how='left')

        df_final['datediff_algo']=df_final['algo_date']-df_final['assess_date']
        df_final['datediff_firstoccur']=df_final['firstoccur_date']-df_final['assess_date']

        datediff_int=[]
        for i in range(len(df_final)):
            datediff_int.append(df_final['datediff_algo'][i].days)

        df_final['datediff_algo_int']=datediff_int

        datediff_int2=[]
        event=[]
        for i in range(len(df_final)):
            if np.isnan(df_final['datediff_algo_int'][i])==True:
                datediff_int2.append(3650)
                event.append(False)
            else:
                if df_final['datediff_algo_int'][i]>3650:
                    datediff_int2.append(3650)
                    event.append(False)
                else:
                    datediff_int2.append(df_final['datediff_algo_int'][i])
                    event.append(True)
        df_final['datediff_algo_int2']=datediff_int2
        df_final['algo_event']=event

        aa=list(df_final['datediff_algo_int2'])
        aa2=[x for x in aa if x < 3650]
        aaa=[x for x in aa if x >=0]
        aaaa=[x for x in aaa if x < 3650]
        exclusion_event_before_assessment=len(aa2)-len(aaaa)
        bb=exclusion_event_before_assessment
        print('algoMACE_within10years=',len(aa2)-bb,'/',upto-bb, ", exclusion_event_before_assessment=",bb)

        txt+= (  'algoMACE_within10years=' + str(len(aa2)-bb) + ' / ' + str(upto-bb)+ " , exclusion_event_before_assessment= "+ str(bb) + '\n'  )


        datediff_int=[]
        for i in range(len(df_final)):
            datediff_int.append(df_final['datediff_firstoccur'][i].days)

        df_final['datediff_firstoccur_int']=datediff_int

        datediff_int2=[]
        event=[]
        for i in range(len(df_final)):
            if np.isnan(df_final['datediff_firstoccur_int'][i])==True:
                datediff_int2.append(3650)
                event.append(False)
            else:
                if df_final['datediff_firstoccur_int'][i]>3650:
                    datediff_int2.append(3650)
                    event.append(False)
                else:
                    datediff_int2.append(df_final['datediff_firstoccur_int'][i])
                    event.append(True)
        df_final['datediff_firstoccur_int2']=datediff_int2
        df_final['firstoccur_event']=event

        aa=list(df_final['datediff_firstoccur_int2'])
        aa2=[x for x in aa if x < 3650]
        aaa=[x for x in aa if x >=0]
        aaaa=[x for x in aaa if x < 3650]
        exclusion_event_before_assessment=len(aa2)-len(aaaa)
        bb=exclusion_event_before_assessment
        print('firstoccurMACE_within10years=',len(aa2)-bb,'/',upto-bb, ", exclusion_event_before_assessment=",bb)

        txt+= (  'firstoccurMACE_within10years=' + str(len(aa2)-bb) + ' / ' + str(upto-bb)+ " , exclusion_event_before_assessment= "+ str(bb) + '\n'  )

        
        ###################    
        df_final=df_final[df_final['datediff_firstoccur_int2']<=3650]
        df_final=df_final[df_final['datediff_firstoccur_int2']>=0]
        df_final.reset_index(inplace=True,drop=True)


        scorefinallist=list(df_final['score_final'])
        t1=np.percentile(scorefinallist,100/3)
        t2=np.percentile(scorefinallist,200/3)
        newcatfinal=[]
        t1=10
        t2=20
        for i in range(len(df_final)):
            if df_final['score_final'][i]<=t1:
                newcatfinal.append(1)
            elif df_final['score_final'][i]<=t2:
                newcatfinal.append(2)
            else:
                newcatfinal.append(3)
        df_final['category_final']=newcatfinal
        print(temper, t1, t2, 'score_final t1 t2')
        t1_gpt=t1
        t2_gpt=t2

        ###################
        
        

        framingham_cat=[]
        for i in range(len(df_final)):
            if df_final['Framingham'][i]<=10:
                framingham_cat.append(1)
            elif df_final['Framingham'][i]<=20:
                framingham_cat.append(2)
            else:
                framingham_cat.append(3)

        df_final['framingham_cat']=framingham_cat

        ACC_AHA_cat=[]
        for i in range(len(df_final)):
            if df_final['ACC/AHA'][i]<=7.5:
                ACC_AHA_cat.append(1)
            elif df_final['ACC/AHA'][i]<=20:
                ACC_AHA_cat.append(2)
            else:
                ACC_AHA_cat.append(3)

        df_final['ACC_AHA_cat']=ACC_AHA_cat

        scorefinallist=list(df_final['Framingham'])
        t1=np.percentile(scorefinallist,100/3)
        t2=np.percentile(scorefinallist,200/3)
        newcatfinal=[]
        t1=10
        t2=20
        for i in range(len(df_final)):
            if df_final['Framingham'][i]<=t1:
                newcatfinal.append(1)
            elif df_final['Framingham'][i]<=t2:
                newcatfinal.append(2)
            else:
                newcatfinal.append(3)
        df_final['framingham_cat']=newcatfinal
        print(temper, t1, t2, 'framingham t1 t2')
        t1_fram=t1
        t2_fram=t2

        scorefinallist=list(df_final['ACC/AHA'])
        t1=np.percentile(scorefinallist,100/3)
        t2=np.percentile(scorefinallist,200/3)
        newcatfinal=[]
        t1=7.5
        t2=20
        for i in range(len(df_final)):
            if df_final['ACC/AHA'][i]<=t1:
                newcatfinal.append(1)
            elif df_final['ACC/AHA'][i]<=t2:
                newcatfinal.append(2)
            else:
                newcatfinal.append(3)
        df_final['ACC_AHA_cat']=newcatfinal
        print(temper, t1, t2, 'PCE t1 t2')
        t1_PCE=t1
        t2_PCE=t2

        t1list=[t1_gpt,t1_fram,t1_PCE]
        t2list=[t2_gpt,t2_fram,t2_PCE]

        t1t2df=pd.DataFrame({'t1':t1list,'t2':t2list})
        t1t2df.index=['gpt','framingham','PCE']
        t1t2df.to_csv(newtablename+'/t1t2.csv')


        score1list=list(df_final['score_final'])
        score2list=list(df_final['Framingham'])
        score3list=list(df_final['ACC/AHA'])

        score1list.sort()
        score2list.sort()
        score3list.sort()

        score1_newcat=[]
        score2_newcat=[]
        score3_newcat=[]



        for i in range(len(df_final)):
            if df_final['score_final'][i]<score1list[round(upto*0.6)]:
                score1_newcat.append(1)
            elif df_final['score_final'][i]<score1list[round(upto*0.9)]:
                score1_newcat.append(2)
            else:
                score1_newcat.append(3)

            if df_final['Framingham'][i]<score2list[round(upto*0.6)]:
                score2_newcat.append(1)
            elif df_final['Framingham'][i]<score2list[round(upto*0.9)]:
                score2_newcat.append(2)
            else:
                score2_newcat.append(3)

            if df_final['ACC/AHA'][i]<score3list[round(upto*0.6)]:
                score3_newcat.append(1)
            elif df_final['ACC/AHA'][i]<score3list[round(upto*0.9)]:
                score3_newcat.append(2)
            else:
                score3_newcat.append(3)

        df_final['score1_newcat']=score1_newcat
        df_final['score2_newcat']=score2_newcat
        df_final['score3_newcat']=score3_newcat

        cutofftxt+= 'allcausemortality, ChatGPT_score, 0.6, 0.9\n'
        cutofftxt+= (str(round(score1list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score1list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'allcausemortality, Framingham, 0.6, 0.9\n'
        cutofftxt+= (str(round(score2list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score2list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'allcausemortality, ACC/AHA, 0.6, 0.9\n'
        cutofftxt+= (str(round(score3list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score3list[round(upto*0.9)],1)) + '\n')


        df_final_temp=df_final[df_final['datediff_algo_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_algo_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)
        score1list=list(df_final_temp['score_final'])
        score2list=list(df_final_temp['Framingham'])
        score3list=list(df_final_temp['ACC/AHA'])

        score1list.sort()
        score2list.sort()
        score3list.sort()

        score1_newcat=[]
        score2_newcat=[]
        score3_newcat=[]

        for i in range(len(df_final_temp)):
            if df_final_temp['score_final'][i]<score1list[round(upto*0.6)]:
                score1_newcat.append(1)
            elif df_final_temp['score_final'][i]<score1list[round(upto*0.9)]:
                score1_newcat.append(2)
            else:
                score1_newcat.append(3)

            if df_final_temp['Framingham'][i]<score2list[round(upto*0.6)]:
                score2_newcat.append(1)
            elif df_final_temp['Framingham'][i]<score2list[round(upto*0.9)]:
                score2_newcat.append(2)
            else:
                score2_newcat.append(3)

            if df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.6)]:
                score3_newcat.append(1)
            elif df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.9)]:
                score3_newcat.append(2)
            else:
                score3_newcat.append(3)

        df_final_temp['score1_newcat_algoMACE']=score1_newcat
        df_final_temp['score2_newcat_algoMACE']=score2_newcat
        df_final_temp['score3_newcat_algoMACE']=score3_newcat
        df_final_temp=df_final_temp[['eid','score1_newcat_algoMACE','score2_newcat_algoMACE','score3_newcat_algoMACE']]
        df_final=pd.merge(df_final, df_final_temp, left_on='eid', right_on='eid', how='left')


        cutofftxt+= 'algoMACE, ChatGPT_score, 0.6, 0.9\n'
        cutofftxt+= (str(round(score1list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score1list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'algoMACE, Framingham, 0.6, 0.9\n'
        cutofftxt+= (str(round(score2list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score2list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'algoMACE, ACC/AHA, 0.6, 0.9\n'
        cutofftxt+= (str(round(score3list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score3list[round(upto*0.9)],1)) + '\n')




        df_final_temp=df_final[df_final['datediff_firstoccur_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_firstoccur_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)

        score1list=list(df_final_temp['score_final'])
        score2list=list(df_final_temp['Framingham'])
        score3list=list(df_final_temp['ACC/AHA'])

        score1list.sort()
        score2list.sort()
        score3list.sort()

        score1_newcat=[]
        score2_newcat=[]
        score3_newcat=[]

        for i in range(len(df_final_temp)):
            if df_final_temp['score_final'][i]<score1list[round(upto*0.6)]:
                score1_newcat.append(1)
            elif df_final_temp['score_final'][i]<score1list[round(upto*0.9)]:
                score1_newcat.append(2)
            else:
                score1_newcat.append(3)

            if df_final_temp['Framingham'][i]<score2list[round(upto*0.6)]:
                score2_newcat.append(1)
            elif df_final_temp['Framingham'][i]<score2list[round(upto*0.9)]:
                score2_newcat.append(2)
            else:
                score2_newcat.append(3)

            if df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.6)]:
                score3_newcat.append(1)
            elif df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.9)]:
                score3_newcat.append(2)
            else:
                score3_newcat.append(3)

        df_final_temp['score1_newcat_firstoccurMACE']=score1_newcat
        df_final_temp['score2_newcat_firstoccurMACE']=score2_newcat
        df_final_temp['score3_newcat_firstoccurMACE']=score3_newcat
        df_final_temp=df_final_temp[['eid','score1_newcat_firstoccurMACE','score2_newcat_firstoccurMACE','score3_newcat_firstoccurMACE']]
        df_final=pd.merge(df_final, df_final_temp, left_on='eid', right_on='eid', how='left')

        cutofftxt+= 'firstoccurMACE, ChatGPT_score, 0.6, 0.9\n'
        cutofftxt+= (str(round(score1list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score1list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'firstoccurMACE, Framingham, 0.6, 0.9\n'
        cutofftxt+= (str(round(score2list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score2list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'firstoccurMACE, ACC/AHA, 0.6, 0.9\n'
        cutofftxt+= (str(round(score3list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score3list[round(upto*0.9)],1)) + '\n')





        try:
            os.mkdir(newtablename)
        except:
            pass


        file = open(newtablename+"/samplenumber.txt", "w") 
        file.write(txt)
        file.close()

        file = open(newtablename+"/60percent90percent_cutoffs.txt", "w") 
        file.write(cutofftxt)
        file.close()

        #df_final=df_final.dropna(subset='score0',axis=0)
        #df_final.reset_index(inplace=True,drop=True)


        ######## 설정변수 ##########
        ylim1_survival_allcausemortality=0.86
        ylim2_survival_allcausemortality=1.01
        ylim1_survival_algoMACE=0.90
        ylim2_survival_algoMACE=1.01
        ylim1_survival_firstoccurMACE=0.78
        ylim2_survival_firstoccurMACE=1.01


        ############################
        txt=''
        txt_men_40=''
        txt_women_40=''
        txt_men_50=''
        txt_women_50=''
        txt_men_60=''
        txt_women_60=''
        txt_men_allages=''
        txt_women_allages=''
        txt_bothsex_40=''
        txt_bothsex_50=''
        txt_bothsex_60=''

        catnamedict={}
        catnamedict['ChatGPT (Original)']='category_final'
        catnamedict['ChatGPT (6:3:1)']='score1_newcat'
        catnamedict['Framingham (Original)']='framingham_cat'
        catnamedict['Framingham (6:3:1)']='score2_newcat'
        catnamedict['ACC/AHA ASCVD (Original)']='ACC_AHA_cat'
        catnamedict['ACC/AHA ASCVD (6:3:1)']='score3_newcat'

        scorenamedict={}
        scorenamedict['ChatGPT (Original)']='score_final'
        #scorenamedict['ChatGPT (Original)']='score0'
        scorenamedict['ChatGPT (6:3:1)']='score_final'
        scorenamedict['Framingham (Original)']='Framingham'
        scorenamedict['Framingham (6:3:1)']='Framingham'
        scorenamedict['ACC/AHA ASCVD (Original)']='ACC/AHA'
        scorenamedict['ACC/AHA ASCVD (6:3:1)']='ACC/AHA'

        outcome_event_dict={}
        outcome_event_dict['allcausemortality']='event'
        outcome_event_dict['algoMACE']='algo_event'
        outcome_event_dict['firstoccurMACE']='firstoccur_event'

        outcome_datediff_dict={}
        outcome_datediff_dict['allcausemortality']='datediff_int2'
        outcome_datediff_dict['algoMACE']='datediff_algo_int2'
        outcome_datediff_dict['firstoccurMACE']='datediff_firstoccur_int2'

        ylim1_dict={}
        ylim1_dict['allcausemortality']=ylim1_survival_allcausemortality
        ylim1_dict['algoMACE']=ylim1_survival_algoMACE
        ylim1_dict['firstoccurMACE']=ylim1_survival_firstoccurMACE

        ylim2_dict={}
        ylim2_dict['allcausemortality']=ylim2_survival_allcausemortality
        ylim2_dict['algoMACE']=ylim2_survival_algoMACE
        ylim2_dict['firstoccurMACE']=ylim2_survival_firstoccurMACE

        def survivalgraph(catname1,catname2,outcome,ylim1,ylim2):
            global txt
            global txt_men_40
            global txt_men_50
            global txt_men_60
            global txt_women_40
            global txt_women_50
            global txt_women_60
            global txt_bothsex_40
            global txt_bothsex_50
            global txt_bothsex_60
            global txt_men_allages
            global txt_women_allages
            datediff_temp=outcome_datediff_dict[outcome]
            event_temp=outcome_event_dict[outcome]
            score_temp=scorenamedict[catname1]

            df_final_temp=df_final[df_final[datediff_temp]<=3650][['age','sex',catname2,datediff_temp,event_temp,score_temp]]
            df_final_temp=df_final_temp[df_final_temp[datediff_temp]>=0]

            #df_final_temp2=df_final_temp[[event_temp,score_temp]]
            #print(len(df_final_temp2))
            #df_final_temp2=df_final_temp2.dropna()
            #fpr,tpr,thres=roc_curve(df_final_temp2[event_temp],df_final_temp2[score_temp])
            #roc_auc = round(auc(fpr, tpr),3)
            #print(len(df_final_temp2),np.sum(df_final_temp2[event_temp]),roc_auc)

            fpr,tpr,thres=roc_curve(df_final_temp[event_temp],df_final_temp[score_temp])
            roc_auc = round(auc(fpr, tpr),3)

            df_final_temp_men_40=df_final_temp[df_final_temp['age']>=40]
            df_final_temp_men_40=df_final_temp_men_40[df_final_temp_men_40['age']<50]
            df_final_temp_men_40=df_final_temp_men_40[df_final_temp_men_40['sex']==1]
            df_final_temp_men_40.reset_index(inplace=True,drop=True)

            df_final_temp_men_50=df_final_temp[df_final_temp['age']>=50]
            df_final_temp_men_50=df_final_temp_men_50[df_final_temp_men_50['age']<60]
            df_final_temp_men_50=df_final_temp_men_50[df_final_temp_men_50['sex']==1]
            df_final_temp_men_50.reset_index(inplace=True,drop=True)

            df_final_temp_men_60=df_final_temp[df_final_temp['age']>=60]
            df_final_temp_men_60=df_final_temp_men_60[df_final_temp_men_60['age']<71]
            df_final_temp_men_60=df_final_temp_men_60[df_final_temp_men_60['sex']==1]
            df_final_temp_men_60.reset_index(inplace=True,drop=True)

            df_final_temp_women_40=df_final_temp[df_final_temp['age']>=40]
            df_final_temp_women_40=df_final_temp_women_40[df_final_temp_women_40['age']<50]
            df_final_temp_women_40=df_final_temp_women_40[df_final_temp_women_40['sex']==0]
            df_final_temp_women_40.reset_index(inplace=True,drop=True)

            df_final_temp_women_50=df_final_temp[df_final_temp['age']>=50]
            df_final_temp_women_50=df_final_temp_women_50[df_final_temp_women_50['age']<60]
            df_final_temp_women_50=df_final_temp_women_50[df_final_temp_women_50['sex']==0]
            df_final_temp_women_50.reset_index(inplace=True,drop=True)

            df_final_temp_women_60=df_final_temp[df_final_temp['age']>=60]
            df_final_temp_women_60=df_final_temp_women_60[df_final_temp_women_60['age']<71]
            df_final_temp_women_60=df_final_temp_women_60[df_final_temp_women_60['sex']==0]
            df_final_temp_women_60.reset_index(inplace=True,drop=True)

            df_final_temp_bothsex_40=df_final_temp[df_final_temp['age']>=40]
            df_final_temp_bothsex_40=df_final_temp_bothsex_40[df_final_temp_bothsex_40['age']<50]
            df_final_temp_bothsex_40.reset_index(inplace=True,drop=True)

            df_final_temp_bothsex_50=df_final_temp[df_final_temp['age']>=50]
            df_final_temp_bothsex_50=df_final_temp_bothsex_50[df_final_temp_bothsex_50['age']<60]
            df_final_temp_bothsex_50.reset_index(inplace=True,drop=True)

            df_final_temp_bothsex_60=df_final_temp[df_final_temp['age']>=60]
            df_final_temp_bothsex_60=df_final_temp_bothsex_60[df_final_temp_bothsex_60['age']<71]
            df_final_temp_bothsex_60.reset_index(inplace=True,drop=True)

            df_final_temp_men_allages=df_final_temp[df_final_temp['sex']==1]
            df_final_temp_men_allages.reset_index(inplace=True,drop=True)

            df_final_temp_women_allages=df_final_temp[df_final_temp['sex']==0]
            df_final_temp_women_allages.reset_index(inplace=True,drop=True)

            fpr,tpr,thres=roc_curve(df_final_temp_men_40[event_temp],df_final_temp_men_40[score_temp])
            roc_auc_men_40 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_men_50[event_temp],df_final_temp_men_50[score_temp])
            roc_auc_men_50 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_men_60[event_temp],df_final_temp_men_60[score_temp])
            roc_auc_men_60 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_40[event_temp],df_final_temp_women_40[score_temp])
            roc_auc_women_40 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_50[event_temp],df_final_temp_women_50[score_temp])
            roc_auc_women_50 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_60[event_temp],df_final_temp_women_60[score_temp])
            roc_auc_women_60 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_bothsex_40[event_temp],df_final_temp_bothsex_40[score_temp])
            roc_auc_bothsex_40 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_bothsex_50[event_temp],df_final_temp_bothsex_50[score_temp])
            roc_auc_bothsex_50 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_bothsex_60[event_temp],df_final_temp_bothsex_60[score_temp])
            roc_auc_bothsex_60 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_men_allages[event_temp],df_final_temp_men_allages[score_temp])
            roc_auc_men_allages = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_allages[event_temp],df_final_temp_women_allages[score_temp])
            roc_auc_women_allages = round(auc(fpr, tpr),3)



            cat1 = df_final_temp[df_final_temp[catname2] == 1]
            cat2 = df_final_temp[df_final_temp[catname2] == 2]
            cat3 = df_final_temp[df_final_temp[catname2] == 3]
            print(outcome,catname2, len(cat1),len(cat2),len(cat3))

            kmf = KaplanMeierFitter()
            kmf.fit(cat1[datediff_temp], cat1[event_temp], label='Low')
            ax_kmf = kmf.plot(loc=slice(0.,3650.))
            kmf.fit(cat2[datediff_temp], cat2[event_temp], label='Moderate')
            ax_kmf = kmf.plot(ax = ax_kmf,loc=slice(0.,3650.))
            kmf.fit(cat3[datediff_temp], cat3[event_temp], label='High')
            ax_kmf = kmf.plot(ax = ax_kmf,loc=slice(0.,3650.))

            ax_kmf.set_ylim([ylim1, ylim2])
            ax_kmf.set_yticks(np.arange(ylim1, ylim2, step=0.02))
            ax_kmf.set_xlabel('Time (days)')
            ax_kmf.set_ylabel('Survival Rate')
            ['ChatGPT (Original)','ChatGPT (6:3:1)','Framingham (Original)','Framingham (6:3:1)' ,'ACC/AHA ASCVD (Original)','ACC/AHA ASCVD (6:3:1)']
            if catname1=='ChatGPT (Original)':
                ax_kmf.set_title('GPT-4')
            elif catname1=='Framingham (Original)':
                ax_kmf.set_title('Framingham')
            elif catname1=='ACC/AHA ASCVD (Original)':
                ax_kmf.set_title('ACC/AHA')
            else:
                ax_kmf.set_title(catname1)



            try:
                os.mkdir(newtablename)
            except:
                pass
            try:
                os.mkdir(newtablename+'/survival')
            except:
                pass
            catname1_temp=catname1.replace(' ','_')
            catname1_temp=catname1_temp.replace('(','_')
            catname1_temp=catname1_temp.replace(')','')
            catname1_temp=catname1_temp.replace(':','')
            catname1_temp=catname1_temp.replace('/','')
            ax_kmf.get_figure().savefig(newtablename+"/survival/"+outcome+'_' +catname1_temp+   ".png",transparent=True)
            plt.close('all')

            if catname1=='ChatGPT (Original)':
                txt+= ('ChatGPT' + ' AUROC = ' + str(roc_auc) +'\n' )
                txt_men_40+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_40) +'\n' )
                txt_men_50+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_50) +'\n' )
                txt_men_60+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_60) +'\n' )
                txt_women_40+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_40) +'\n' )
                txt_women_50+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_50) +'\n' )
                txt_women_60+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_60) +'\n' )
                txt_bothsex_40+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_bothsex_40) +'\n' )
                txt_bothsex_50+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_bothsex_50) +'\n' )
                txt_bothsex_60+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_bothsex_60) +'\n' )
                txt_men_allages+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_allages) +'\n' )
                txt_women_allages+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_allages) +'\n' )
            elif catname1=='Framingham (Original)':
                txt+= ('Framingham' + ' AUROC = ' + str(roc_auc) +'\n' )
                txt_men_40+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_40) +'\n' )
                txt_men_50+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_50) +'\n' )
                txt_men_60+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_60) +'\n' )
                txt_women_40+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_40) +'\n' )
                txt_women_50+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_50) +'\n' )
                txt_women_60+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_60) +'\n' )
                txt_bothsex_40+= ('Framingham' + ' AUROC = ' + str(roc_auc_bothsex_40) +'\n' )
                txt_bothsex_50+= ('Framingham' + ' AUROC = ' + str(roc_auc_bothsex_50) +'\n' )
                txt_bothsex_60+= ('Framingham' + ' AUROC = ' + str(roc_auc_bothsex_60) +'\n' )
                txt_men_allages+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_allages) +'\n' )
                txt_women_allages+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_allages) +'\n' )
            elif catname1=='ACC/AHA ASCVD (Original)':
                txt+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc) +'\n' )
                txt_men_40+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_40) +'\n' )
                txt_men_50+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_50) +'\n' )
                txt_men_60+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_60) +'\n' )
                txt_women_40+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_40) +'\n' )
                txt_women_50+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_50) +'\n' )
                txt_women_60+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_60) +'\n' )
                txt_bothsex_40+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_bothsex_40) +'\n' )
                txt_bothsex_50+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_bothsex_50) +'\n' )
                txt_bothsex_60+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_bothsex_60) +'\n' )
                txt_men_allages+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_allages) +'\n' )
                txt_women_allages+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_allages) +'\n' )
            else:
                pass




        for outcome in ['allcausemortality','algoMACE','firstoccurMACE']:
            count=0
            for catname1 in ['ChatGPT (Original)','ChatGPT (6:3:1)','Framingham (Original)','Framingham (6:3:1)' ,'ACC/AHA ASCVD (Original)','ACC/AHA ASCVD (6:3:1)']:
                if count==0:
                    txt+='===== ' +outcome+' ====='+'\n'
                    txt_men_40+='===== ' +outcome+' ====='+'\n'
                    txt_men_50+='===== ' +outcome+' ====='+'\n'
                    txt_men_60+='===== ' +outcome+' ====='+'\n'
                    txt_women_40+='===== ' +outcome+' ====='+'\n'
                    txt_women_50+='===== ' +outcome+' ====='+'\n'
                    txt_women_60+='===== ' +outcome+' ====='+'\n'
                    txt_bothsex_40+='===== ' +outcome+' ====='+'\n'
                    txt_bothsex_50+='===== ' +outcome+' ====='+'\n'
                    txt_bothsex_60+='===== ' +outcome+' ====='+'\n'
                    txt_men_allages+='===== ' +outcome+' ====='+'\n'
                    txt_women_allages+='===== ' +outcome+' ====='+'\n'
                    count+=1

                ylim1_survival=ylim1_dict[outcome]
                ylim2_survival=ylim2_dict[outcome]
                catname2=catnamedict[catname1]
                survivalgraph(catname1,catname2,outcome,ylim1_survival,ylim2_survival)


        file = open(newtablename+"/AUROC.txt", "w") 
        file.write(txt)
        file.close()

        file = open(newtablename+"/AUROC_men_40.txt", "w") 
        file.write(txt_men_40)
        file.close()

        file = open(newtablename+"/AUROC_men_50.txt", "w") 
        file.write(txt_men_50)
        file.close()

        file = open(newtablename+"/AUROC_men_60.txt", "w") 
        file.write(txt_men_60)
        file.close()

        file = open(newtablename+"/AUROC_women_40.txt", "w") 
        file.write(txt_women_40)
        file.close()

        file = open(newtablename+"/AUROC_women_50.txt", "w") 
        file.write(txt_women_50)
        file.close()

        file = open(newtablename+"/AUROC_women_60.txt", "w") 
        file.write(txt_women_60)
        file.close()

        file = open(newtablename+"/AUROC_bothsex_40.txt", "w") 
        file.write(txt_bothsex_40)
        file.close()

        file = open(newtablename+"/AUROC_bothsex_50.txt", "w") 
        file.write(txt_bothsex_50)
        file.close()

        file = open(newtablename+"/AUROC_bothsex_60.txt", "w") 
        file.write(txt_bothsex_60)
        file.close()

        file = open(newtablename+"/AUROC_men_allages.txt", "w") 
        file.write(txt_men_allages)
        file.close()

        file = open(newtablename+"/AUROC_women_allages.txt", "w") 
        file.write(txt_women_allages)
        file.close()

        ######## 설정변수 ##########
        lim=80


        ############################


        txt=''

        def scatter_plot(xxx,yyy,lim):
            global txt
            plt.figure(figsize=(10,10))
            if xxx=='ChatGPT':
                xxxx='GPT-4'
            else:
                xxxx=xxx
            if yyy=='ACC/AHA':
                yyyy='ACC/AHA'
            else:
                yyyy=yyy





            plt.title(xxxx+ ' VS '+ yyyy , fontsize=20)

            if xxx=='ChatGPT':
                plt.xlabel('GPT-4', fontsize=14)
            else:
                plt.xlabel(xxx, fontsize=14)
            if yyy=='ACC/AHA':
                plt.ylabel('ACC/AHA', fontsize=14)
            else:
                plt.ylabel(yyy, fontsize=14)

            plt.xlim([-5, lim])
            plt.ylim([-5, lim])
            plt.yticks(np.arange(0, lim+10, step=10))
            plt.xticks(np.arange(0, lim+10, step=10))
            if xxx=='ChatGPT':
                xxx_column='score_final'
            elif xxx=='Framingham':
                xxx_column='Framingham'
            elif xxx=='ACC/AHA':
                xxx_column='ACC/AHA'
            else:
                print('error')

            if yyy=='ChatGPT':
                yyy_column='score_final'
            elif yyy=='Framingham':
                yyy_column='Framingham'
            elif yyy=='ACC/AHA':
                yyy_column='ACC/AHA'
            else:
                print('error')    

            try:
                os.mkdir(newtablename)
            except:
                pass
            try:
                os.mkdir(newtablename+'/scatter')
            except:
                pass

            xxx=xxx.replace('/','_')
            yyy=yyy.replace('/','_')

            plt.scatter(df_final[xxx_column],df_final[yyy_column],s=0.08, alpha=0.8)
            plt.savefig(newtablename+"/scatter/"+xxx+'_'+yyy+".png",transparent=True)
            plt.close('all')

            pearr=round(pearsonr(df_final[xxx_column],df_final[yyy_column]).statistic,3)
            pearp=round(pearsonr(df_final[xxx_column],df_final[yyy_column]).pvalue,3)

            txt+='==========================\n'
            txt+=('--- '+xxx+ ' VS '+ yyy + ' ---'+'\n')
            txt+=('pearson r = ' + str(pearr)+', pearson pvalue = '+str(pearp)   + '\n')
            txt+=(   xxx +'_mean = '+ str(round(np.mean(df_final[xxx_column]),3))   + '\n')
            txt+=(   yyy +'_mean = '+ str(round(np.mean(df_final[yyy_column]),3))   + '\n')
            txt+=(   xxx +'_shapiro_pvalue = '+ str(round(shapiro(df_final[xxx_column]).pvalue,3))   + '\n')
            txt+=(   yyy +'_shapiro_pvalue = '+ str(round(shapiro(df_final[yyy_column]).pvalue,3))   + '\n')
            txt+=(   'mannwhitneyu_pvalue = '+ str(round(mwu(df_final[xxx_column],df_final[yyy_column]).pvalue,3))   + '\n')



        for xxx,yyy in [['ChatGPT','Framingham'],['ChatGPT','ACC/AHA'],['Framingham','ACC/AHA']]:
            scatter_plot(xxx,yyy,lim)

        txt+='=========================='

        file = open(newtablename+"/scatter/"+'stats.txt', "w") 
        file.write(txt)
        file.close()



        #######################################################

        print(list(df_final.columns))

        score_individuals=[]
        for sss in range(iterations):
            score_individuals.append('score'+str(sss))
        df_final_temp=df_final[df_final['datediff_algo_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_algo_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)
        Framingham_temp_algo=df_final_temp['Framingham']
        df_final_temp_algo_scores=df_final_temp[score_individuals]

        df_final_temp=df_final[df_final['datediff_firstoccur_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_firstoccur_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)
        Framingham_temp_firstoccur=df_final_temp['Framingham']
        df_final_temp_firstoccur_scores=df_final_temp[score_individuals]

        mean=df_final_temp_firstoccur_scores.mean(axis=1)
        std=df_final_temp_firstoccur_scores.std(axis=1)
        df_final_temp_firstoccur_scores['mean']=mean
        df_final_temp_firstoccur_scores['std']=std
        df_final_temp_firstoccur_scores['CV']=df_final_temp_firstoccur_scores['std']/df_final_temp_firstoccur_scores['mean']

        df_final_temp_firstoccur_scores['upper']=df_final_temp_firstoccur_scores['mean']+df_final_temp_firstoccur_scores['std']
        df_final_temp_firstoccur_scores['under']=df_final_temp_firstoccur_scores['mean']-df_final_temp_firstoccur_scores['std']
        df_final_temp_firstoccur_scores['Framingham']=Framingham_temp_firstoccur


        scorefinallist=list(df_final['score_final'])
        t1=np.percentile(scorefinallist,100/3)
        t2=np.percentile(scorefinallist,200/3)
        t1=10
        t2=20
        newcatfinal=[]
        for i in range(len(df_final)):
            if df_final['score_final'][i]<=t1:
                newcatfinal.append(1)
            elif df_final['score_final'][i]<=t2:
                newcatfinal.append(2)
            else:
                newcatfinal.append(3)
        df_final['category_final']=newcatfinal
        
        
        
        df_final_temp_firstoccur_scores_low=df_final_temp_firstoccur_scores[df_final_temp_firstoccur_scores['mean']<=t1].reset_index(drop=True)
        df_final_temp_firstoccur_scores_mod=df_final_temp_firstoccur_scores[df_final_temp_firstoccur_scores['mean']<=t2].reset_index(drop=True)
        df_final_temp_firstoccur_scores_mod=df_final_temp_firstoccur_scores_mod[df_final_temp_firstoccur_scores_mod['mean']>t1].reset_index(drop=True)
        df_final_temp_firstoccur_scores_hig=df_final_temp_firstoccur_scores[df_final_temp_firstoccur_scores['mean']>t2].reset_index(drop=True)

        firstoccur_total_CV=np.mean(df_final_temp_firstoccur_scores['CV'])
        firstoccur_total_sd=np.mean(df_final_temp_firstoccur_scores['std'])
        firstoccur_low_CV=np.mean(df_final_temp_firstoccur_scores_low['CV'])
        firstoccur_mod_CV=np.mean(df_final_temp_firstoccur_scores_mod['CV'])
        firstoccur_hig_CV=np.mean(df_final_temp_firstoccur_scores_hig['CV'])
        firstoccur_low_sd=np.mean(df_final_temp_firstoccur_scores_low['std'])
        firstoccur_mod_sd=np.mean(df_final_temp_firstoccur_scores_mod['std'])
        firstoccur_hig_sd=np.mean(df_final_temp_firstoccur_scores_hig['std'])






        mean=df_final_temp_algo_scores.mean(axis=1)
        std=df_final_temp_algo_scores.std(axis=1)
        df_final_temp_algo_scores['mean']=mean
        df_final_temp_algo_scores['std']=std
        df_final_temp_algo_scores['CV']=df_final_temp_algo_scores['std']/df_final_temp_algo_scores['mean']

        df_final_temp_algo_scores['upper']=df_final_temp_algo_scores['mean']+df_final_temp_algo_scores['std']
        df_final_temp_algo_scores['under']=df_final_temp_algo_scores['mean']-df_final_temp_algo_scores['std']
        df_final_temp_algo_scores['Framingham']=Framingham_temp_algo

        df_final_temp_algo_scores_low=df_final_temp_algo_scores[df_final_temp_algo_scores['mean']<=t1].reset_index(drop=True)
        df_final_temp_algo_scores_mod=df_final_temp_algo_scores[df_final_temp_algo_scores['mean']<=t2].reset_index(drop=True)
        df_final_temp_algo_scores_mod=df_final_temp_algo_scores_mod[df_final_temp_algo_scores_mod['mean']>t1].reset_index(drop=True)
        df_final_temp_algo_scores_hig=df_final_temp_algo_scores[df_final_temp_algo_scores['mean']>t2].reset_index(drop=True)

        algo_total_CV=np.mean(df_final_temp_algo_scores['CV'])
        algo_total_sd=np.mean(df_final_temp_algo_scores['std'])
        algo_low_CV=np.mean(df_final_temp_algo_scores_low['CV'])
        algo_mod_CV=np.mean(df_final_temp_algo_scores_mod['CV'])
        algo_hig_CV=np.mean(df_final_temp_algo_scores_hig['CV'])
        algo_low_sd=np.mean(df_final_temp_algo_scores_low['std'])
        algo_mod_sd=np.mean(df_final_temp_algo_scores_mod['std'])
        algo_hig_sd=np.mean(df_final_temp_algo_scores_hig['std'])

        firstoccurcvsdlist=[firstoccur_total_CV,firstoccur_total_sd,firstoccur_low_CV,firstoccur_mod_CV,firstoccur_hig_CV,firstoccur_low_sd,firstoccur_mod_sd,firstoccur_hig_sd]
        algocvsdlist=[algo_total_CV,algo_total_sd,algo_low_CV,algo_mod_CV,algo_hig_CV,algo_low_sd,algo_mod_sd,algo_hig_sd]


        cvsddf=pd.DataFrame({'algo':algocvsdlist,'firstoccur':firstoccurcvsdlist})
        cvsddf.index=['total_CV','total_SD','low_CV','mod_CV','high_CV','low_SD','mod_SD','high_SD']
        cvsddf.to_csv(newtablename+'/CV_SD.csv')




        df_final.to_csv('20230825_df_final_gpt4_50000_5iter_nottertile.csv',index=False)


        #######################################################

        df_final['BMI']=(df_final['weight']/(df_final['height']/100))/(df_final['height']/100)


        dfdfdf2=df_final[['age','sex','total_chol','HDL','LDL','TG','SBP','DBP','BMI','Smoking','med_final','DM_final','Framingham','ACC/AHA','framingham_cat','ACC_AHA_cat','category_final','score_final','firstoccur_event']]
        dfdfdf2.columns=['Age','Sex','Total_chol','HDL','LDL','TG','SBP','DBP','BMI','Smoking','BP medication','DM','Framingham','ACC/AHA','Framingham category','ACC/AHA category','ChatGPT category','ChatGPT score','firstoccur_event']

        catcolumns=['Smoking','Sex','DM','BP medication','ChatGPT category','Framingham category','ACC/AHA category','firstoccur_event']

        def table1(catcolumn):
            nonnormallist=[]
            def _normality(self, x):
                #print(x.name)

                if len(x.values[~np.isnan(x.values)]) >= 20:
                    p = stats.shapiro(x.values).pvalue
                else:
                    p = None
                # dropna=False argument in pivot_table does not function as expected
                # return -1 instead of None
                if pd.isnull(p):
                    return -1
                if p<=0.05:
                    nonnormallist.append(x.name)
                return p

            TableOne._normality=_normality

            def my_custom_test(group1, group2):
                """
                Hypothesis test for test_self_defined_statistical_tests
                """
                my_custom_test.__name__ = "mannwhitneyu"
                _, pval= scipy.stats.mannwhitneyu(group1, group2)
                return pval

            nonnormallist=[]
            def _normality(self, x):
                #print(x.name)

                if len(x.values[~np.isnan(x.values)]) >= 20:
                    p = stats.shapiro(x.values).pvalue
                else:
                    p = None
                # dropna=False argument in pivot_table does not function as expected
                # return -1 instead of None
                if pd.isnull(p):
                    return -1
                if p<=0.05:
                    nonnormallist.append(x.name)
                return p

            TableOne._normality=_normality

            table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True)
            nonnormallist=list(set(nonnormallist))
            nonnormallist

            table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True,nonnormal=nonnormallist)
            try:
                os.mkdir(newtablename)
            except:
                pass
            try:
                os.mkdir(newtablename+'/table1')
            except:
                pass
            catcolumn=catcolumn.replace(' ','_')
            catcolumn=catcolumn.replace('/','_')
            table1.to_html(newtablename+'/table1/'+catcolumn+'.html')

        for catcolumn in ['ChatGPT category','Framingham category','ACC/AHA category']:
            table1(catcolumn)

In [ ]:
############################
table_name="" #enter table name
iterations=5
upto=50000
############################

for plusplus in range(1): 
    for temper in [0.4]:


        txt=''

        cutofftxt=''
        for i in range(iterations):
            newtablename=table_name + '_' + str(int(temper*10)) + '_' + str(i+plusplus)

            sql_statement="select * from ["+ newtablename + "]"
            conn = pymssql.connect(host=r"(local)", database='WorkDB4', charset='utf8')
            globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)
            globals()['data{}'.format(i)]['rank_by_eid']=globals()['data{}'.format(i)].groupby('eid')['system'].rank(method='first')
            globals()['data{}'.format(i)]=globals()['data{}'.format(i)][globals()['data{}'.format(i)]['rank_by_eid']==1]
            globals()['data{}'.format(i)].reset_index(inplace=True,drop=True)
            globals()['data{}'.format(i)].drop(['rank_by_eid'],axis=1,inplace=True)


        for j in range(iterations):
            dat=globals()['data{}'.format(j)]

            eid=dat['eid']
            score_gpt=dat['score']
            category=dat['category']
            score_framingham=dat['framingham']
            score_acc_aha=dat['ACC_AHA']

            score_gpt2=[]

            for i in range(len(score_gpt)):
                try:
                    score_gpt2.append(float(re.findall("\d+[.]\d+[%]",score_gpt[i])[0].split('%')[0]))
                except:
                    try:
                        score_gpt2.append(float(re.findall("\d+[%]",score_gpt[i])[0].split('%')[0]))
                    except:
                        try:
                            score_gpt2.append(float(re.findall("\d+[.]\d+",score_gpt[i])[0]))
                        except:
                            try:
                                score_gpt2.append(float(re.findall("\d+",score_gpt[i])[-1]))
                            except:
                                score_gpt2.append(float(dat['ACC_AHA'][i]))
                                print(score_gpt[i],'====')


            category2=[]
            for i in range(len(score_gpt2)):
                if np.isnan(score_gpt2[i])==False:
                    if score_gpt2[i]>20:
                        category2.append('High')
                    elif score_gpt2[i]>10:
                        category2.append('Moderate')
                    else:
                        category2.append('Low')
                else:
                    category2.append(np.nan)






            columnname_score='score'+str(j)
            columnname_cat='category'+str(j)
            globals()['data{}_1'.format(j)]=pd.DataFrame({'eid':eid,columnname_score:score_gpt2,columnname_cat:category2})
            globals()['data{}_1'.format(j)]=globals()['data{}_1'.format(j)].dropna(axis=0)
            globals()['data{}_1'.format(j)].reset_index(inplace=True,drop=True)

        df2_0=df.iloc[:upto]

        for i in range(iterations):
            globals()['data{}_1'.format(i)] = globals()['data{}_1'.format(i)].astype({ 'eid' : 'int' })

        for i in range(iterations):
            globals()['df2_{}'.format(i+1)] = pd.merge(globals()['df2_{}'.format(i)], globals()['data{}_1'.format(i)], left_on='eid', right_on='eid', how='left')

        globals()['df2_{}'.format(1)].drop_duplicates(inplace=True,ignore_index=True)
        print("length",len(globals()['df2_{}'.format(1)]))

        for j in range(iterations):
            globals()['category{}_int'.format(j)]=[]

        for j in range(iterations):
            columnname='category'+str(j)
            for i in range(len(globals()['df2_{}'.format(iterations)])):
                if globals()['df2_{}'.format(iterations)][columnname][i]=='Low':
                    globals()['category{}_int'.format(j)].append(1)
                elif globals()['df2_{}'.format(iterations)][columnname][i]=='Moderate':
                    globals()['category{}_int'.format(j)].append(2)
                else:
                    globals()['category{}_int'.format(j)].append(3)

        df_final=globals()['df2_{}'.format(iterations)]

        for j in range(iterations):
            columnname='category'+str(j)+'_int'
            df_final[columnname]=globals()['category{}_int'.format(j)]

        score_final=[]
        category_final=[]
        for i in range(len(df_final)):
            score_temp=[]
            category_temp=[]
            for j in range(iterations):
                columnname1='score'+str(j)
                columname2='category'+str(j)+'_int'
                score_temp.append(df_final[columnname1][i])
                category_temp.append(df_final[columname2][i])

            score_temptemp=np.nanmean(score_temp)
            category_temptemp=np.nanmedian(category_temp)
            score_final.append(score_temptemp)
            category_final.append(category_temptemp)

        df_final['score_final']=score_final
        df_final['category_final']=category_final

        cig_final=[]
        for i in range(len(df_final)):
            if df_final['smoking'][i]==2:
                cig_final.append(1)
            else:
                cig_final.append(0)
        df_final['Smoking']=cig_final

        df_final['death_date']=pd.to_datetime(df_final['death_date'])
        df_final['assess_date']=pd.to_datetime(df_final['assess_date'])
        df_final['datediff']=df_final['death_date']-df_final['assess_date']

        datediff_int=[]
        for i in range(len(df_final)):
            datediff_int.append(df_final['datediff'][i].days)

        df_final['datediff_int']=datediff_int

        datediff_int2=[]
        event=[]
        for i in range(len(df_final)):
            if np.isnan(df_final['datediff_int'][i])==True:
                datediff_int2.append(3650)
                event.append(False)
            else:
                if df_final['datediff_int'][i]>3650:
                    datediff_int2.append(3650)
                    event.append(False)
                else:
                    datediff_int2.append(df_final['datediff_int'][i])
                    event.append(True)
        df_final['datediff_int2']=datediff_int2
        df_final['event']=event

        aa=list(df_final['datediff_int2'])
        aa2=[x for x in aa if x < 3650]
        aaa=[x for x in aa if x >=0]
        aaaa=[x for x in aaa if x < 3650]
        exclusion_event_before_assessment=len(aa2)-len(aaaa)
        bb=exclusion_event_before_assessment

        print('allcausemortality_within10years=',len(aa2)-bb,'/',upto-bb, ", exclusion_event_before_assessment=",bb)

        txt+= (  'allcausemortality_within10years=' + str(len(aa2)-bb) + ' / ' + str(upto-bb)+ " , exclusion_event_before_assessment= "+ str(bb) + '\n'  )

        algo=pd.read_csv('ukb_algorithmic_outcome.csv')
        algo=algo[['eid','Min_Date']]
        algo.columns=['eid','algo_date']
        algo['algo_date']=pd.to_datetime(algo['algo_date'])

        firstoccur=pd.read_csv('ukb_new_add_first_occur.csv')
        firstoccur=firstoccur[['eid','Min_Date']]
        firstoccur.columns=['eid','firstoccur_date']
        firstoccur['firstoccur_date']=pd.to_datetime(firstoccur['firstoccur_date'])

        df_final=pd.merge(df_final, algo, left_on='eid', right_on='eid', how='left')
        df_final=pd.merge(df_final, firstoccur, left_on='eid', right_on='eid', how='left')

        df_final['datediff_algo']=df_final['algo_date']-df_final['assess_date']
        df_final['datediff_firstoccur']=df_final['firstoccur_date']-df_final['assess_date']

        datediff_int=[]
        for i in range(len(df_final)):
            datediff_int.append(df_final['datediff_algo'][i].days)

        df_final['datediff_algo_int']=datediff_int

        datediff_int2=[]
        event=[]
        for i in range(len(df_final)):
            if np.isnan(df_final['datediff_algo_int'][i])==True:
                datediff_int2.append(3650)
                event.append(False)
            else:
                if df_final['datediff_algo_int'][i]>3650:
                    datediff_int2.append(3650)
                    event.append(False)
                else:
                    datediff_int2.append(df_final['datediff_algo_int'][i])
                    event.append(True)
        df_final['datediff_algo_int2']=datediff_int2
        df_final['algo_event']=event

        aa=list(df_final['datediff_algo_int2'])
        aa2=[x for x in aa if x < 3650]
        aaa=[x for x in aa if x >=0]
        aaaa=[x for x in aaa if x < 3650]
        exclusion_event_before_assessment=len(aa2)-len(aaaa)
        bb=exclusion_event_before_assessment
        print('algoMACE_within10years=',len(aa2)-bb,'/',upto-bb, ", exclusion_event_before_assessment=",bb)

        txt+= (  'algoMACE_within10years=' + str(len(aa2)-bb) + ' / ' + str(upto-bb)+ " , exclusion_event_before_assessment= "+ str(bb) + '\n'  )


        datediff_int=[]
        for i in range(len(df_final)):
            datediff_int.append(df_final['datediff_firstoccur'][i].days)

        df_final['datediff_firstoccur_int']=datediff_int

        datediff_int2=[]
        event=[]
        for i in range(len(df_final)):
            if np.isnan(df_final['datediff_firstoccur_int'][i])==True:
                datediff_int2.append(3650)
                event.append(False)
            else:
                if df_final['datediff_firstoccur_int'][i]>3650:
                    datediff_int2.append(3650)
                    event.append(False)
                else:
                    datediff_int2.append(df_final['datediff_firstoccur_int'][i])
                    event.append(True)
        df_final['datediff_firstoccur_int2']=datediff_int2
        df_final['firstoccur_event']=event

        aa=list(df_final['datediff_firstoccur_int2'])
        aa2=[x for x in aa if x < 3650]
        aaa=[x for x in aa if x >=0]
        aaaa=[x for x in aaa if x < 3650]
        exclusion_event_before_assessment=len(aa2)-len(aaaa)
        bb=exclusion_event_before_assessment
        print('firstoccurMACE_within10years=',len(aa2)-bb,'/',upto-bb, ", exclusion_event_before_assessment=",bb)

        txt+= (  'firstoccurMACE_within10years=' + str(len(aa2)-bb) + ' / ' + str(upto-bb)+ " , exclusion_event_before_assessment= "+ str(bb) + '\n'  )


        framingham_cat=[]
        for i in range(len(df_final)):
            if df_final['Framingham'][i]<=10:
                framingham_cat.append(1)
            elif df_final['Framingham'][i]<=20:
                framingham_cat.append(2)
            else:
                framingham_cat.append(3)

        df_final['framingham_cat']=framingham_cat

        ACC_AHA_cat=[]
        for i in range(len(df_final)):
            if df_final['ACC/AHA'][i]<=7.5:
                ACC_AHA_cat.append(1)
            elif df_final['ACC/AHA'][i]<=20:
                ACC_AHA_cat.append(2)
            else:
                ACC_AHA_cat.append(3)

        df_final['ACC_AHA_cat']=ACC_AHA_cat

        score1list=list(df_final['score_final'])
        score2list=list(df_final['Framingham'])
        score3list=list(df_final['ACC/AHA'])

        score1list.sort()
        score2list.sort()
        score3list.sort()

        score1_newcat=[]
        score2_newcat=[]
        score3_newcat=[]



        for i in range(len(df_final)):
            if df_final['score_final'][i]<score1list[round(upto*0.6)]:
                score1_newcat.append(1)
            elif df_final['score_final'][i]<score1list[round(upto*0.9)]:
                score1_newcat.append(2)
            else:
                score1_newcat.append(3)

            if df_final['Framingham'][i]<score2list[round(upto*0.6)]:
                score2_newcat.append(1)
            elif df_final['Framingham'][i]<score2list[round(upto*0.9)]:
                score2_newcat.append(2)
            else:
                score2_newcat.append(3)

            if df_final['ACC/AHA'][i]<score3list[round(upto*0.6)]:
                score3_newcat.append(1)
            elif df_final['ACC/AHA'][i]<score3list[round(upto*0.9)]:
                score3_newcat.append(2)
            else:
                score3_newcat.append(3)

        df_final['score1_newcat']=score1_newcat
        df_final['score2_newcat']=score2_newcat
        df_final['score3_newcat']=score3_newcat

        cutofftxt+= 'allcausemortality, ChatGPT_score, 0.6, 0.9\n'
        cutofftxt+= (str(round(score1list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score1list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'allcausemortality, Framingham, 0.6, 0.9\n'
        cutofftxt+= (str(round(score2list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score2list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'allcausemortality, ACC/AHA, 0.6, 0.9\n'
        cutofftxt+= (str(round(score3list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score3list[round(upto*0.9)],1)) + '\n')


        df_final_temp=df_final[df_final['datediff_algo_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_algo_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)
        df_final_temp.to_csv('20230818_50000_5iter2_finaldf_algo.csv',index=False)
        score1list=list(df_final_temp['score_final'])
        score2list=list(df_final_temp['Framingham'])
        score3list=list(df_final_temp['ACC/AHA'])

        score1list.sort()
        score2list.sort()
        score3list.sort()

        score1_newcat=[]
        score2_newcat=[]
        score3_newcat=[]

        for i in range(len(df_final_temp)):
            if df_final_temp['score_final'][i]<score1list[round(upto*0.6)]:
                score1_newcat.append(1)
            elif df_final_temp['score_final'][i]<score1list[round(upto*0.9)]:
                score1_newcat.append(2)
            else:
                score1_newcat.append(3)

            if df_final_temp['Framingham'][i]<score2list[round(upto*0.6)]:
                score2_newcat.append(1)
            elif df_final_temp['Framingham'][i]<score2list[round(upto*0.9)]:
                score2_newcat.append(2)
            else:
                score2_newcat.append(3)

            if df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.6)]:
                score3_newcat.append(1)
            elif df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.9)]:
                score3_newcat.append(2)
            else:
                score3_newcat.append(3)

        df_final_temp['score1_newcat_algoMACE']=score1_newcat
        df_final_temp['score2_newcat_algoMACE']=score2_newcat
        df_final_temp['score3_newcat_algoMACE']=score3_newcat
        df_final_temp=df_final_temp[['eid','score1_newcat_algoMACE','score2_newcat_algoMACE','score3_newcat_algoMACE']]
        df_final=pd.merge(df_final, df_final_temp, left_on='eid', right_on='eid', how='left')


        cutofftxt+= 'algoMACE, ChatGPT_score, 0.6, 0.9\n'
        cutofftxt+= (str(round(score1list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score1list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'algoMACE, Framingham, 0.6, 0.9\n'
        cutofftxt+= (str(round(score2list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score2list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'algoMACE, ACC/AHA, 0.6, 0.9\n'
        cutofftxt+= (str(round(score3list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score3list[round(upto*0.9)],1)) + '\n')





        df_final_temp=df_final[df_final['datediff_firstoccur_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_firstoccur_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)
        df_final_temp.to_csv('20230818_50000_5iter2_finaldf_firstoccur.csv',index=False)
        score1list=list(df_final_temp['score_final'])
        score2list=list(df_final_temp['Framingham'])
        score3list=list(df_final_temp['ACC/AHA'])

        score1list.sort()
        score2list.sort()
        score3list.sort()

        score1_newcat=[]
        score2_newcat=[]
        score3_newcat=[]

        for i in range(len(df_final_temp)):
            if df_final_temp['score_final'][i]<score1list[round(upto*0.6)]:
                score1_newcat.append(1)
            elif df_final_temp['score_final'][i]<score1list[round(upto*0.9)]:
                score1_newcat.append(2)
            else:
                score1_newcat.append(3)

            if df_final_temp['Framingham'][i]<score2list[round(upto*0.6)]:
                score2_newcat.append(1)
            elif df_final_temp['Framingham'][i]<score2list[round(upto*0.9)]:
                score2_newcat.append(2)
            else:
                score2_newcat.append(3)

            if df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.6)]:
                score3_newcat.append(1)
            elif df_final_temp['ACC/AHA'][i]<score3list[round(upto*0.9)]:
                score3_newcat.append(2)
            else:
                score3_newcat.append(3)

        df_final_temp['score1_newcat_firstoccurMACE']=score1_newcat
        df_final_temp['score2_newcat_firstoccurMACE']=score2_newcat
        df_final_temp['score3_newcat_firstoccurMACE']=score3_newcat
        df_final_temp=df_final_temp[['eid','score1_newcat_firstoccurMACE','score2_newcat_firstoccurMACE','score3_newcat_firstoccurMACE']]
        df_final=pd.merge(df_final, df_final_temp, left_on='eid', right_on='eid', how='left')

        cutofftxt+= 'firstoccurMACE, ChatGPT_score, 0.6, 0.9\n'
        cutofftxt+= (str(round(score1list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score1list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'firstoccurMACE, Framingham, 0.6, 0.9\n'
        cutofftxt+= (str(round(score2list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score2list[round(upto*0.9)],1)) + '\n')
        cutofftxt+= 'firstoccurMACE, ACC/AHA, 0.6, 0.9\n'
        cutofftxt+= (str(round(score3list[round(upto*0.6)],1)) + '\n')
        cutofftxt+= (str(round(score3list[round(upto*0.9)],1)) + '\n')



        try:
            os.mkdir(newtablename)
        except:
            pass


        file = open(newtablename+"/samplenumber.txt", "w") 
        file.write(txt)
        file.close()

        file = open(newtablename+"/60percent90percent_cutoffs.txt", "w") 
        file.write(cutofftxt)
        file.close()

        #df_final=df_final.dropna(subset='score0',axis=0)
        #df_final.reset_index(inplace=True,drop=True)


        ######## 설정변수 ##########
        ylim1_survival_allcausemortality=0.86
        ylim2_survival_allcausemortality=1.01
        ylim1_survival_algoMACE=0.90
        ylim2_survival_algoMACE=1.01
        ylim1_survival_firstoccurMACE=0.82
        ylim2_survival_firstoccurMACE=1.01


        ############################
        txt=''
        txt_men_40=''
        txt_women_40=''
        txt_men_50=''
        txt_women_50=''
        txt_men_60=''
        txt_women_60=''
        txt_men_allages=''
        txt_women_allages=''
        txt_bothsex_40=''
        txt_bothsex_50=''
        txt_bothsex_60=''

        catnamedict={}
        catnamedict['ChatGPT (Original)']='category_final'
        catnamedict['ChatGPT (6:3:1)']='score1_newcat'
        catnamedict['Framingham (Original)']='framingham_cat'
        catnamedict['Framingham (6:3:1)']='score2_newcat'
        catnamedict['ACC/AHA ASCVD (Original)']='ACC_AHA_cat'
        catnamedict['ACC/AHA ASCVD (6:3:1)']='score3_newcat'

        scorenamedict={}
        scorenamedict['ChatGPT (Original)']='score_final'
        #scorenamedict['ChatGPT (Original)']='score0'
        scorenamedict['ChatGPT (6:3:1)']='score_final'
        scorenamedict['Framingham (Original)']='Framingham'
        scorenamedict['Framingham (6:3:1)']='Framingham'
        scorenamedict['ACC/AHA ASCVD (Original)']='ACC/AHA'
        scorenamedict['ACC/AHA ASCVD (6:3:1)']='ACC/AHA'

        outcome_event_dict={}
        outcome_event_dict['allcausemortality']='event'
        outcome_event_dict['algoMACE']='algo_event'
        outcome_event_dict['firstoccurMACE']='firstoccur_event'

        outcome_datediff_dict={}
        outcome_datediff_dict['allcausemortality']='datediff_int2'
        outcome_datediff_dict['algoMACE']='datediff_algo_int2'
        outcome_datediff_dict['firstoccurMACE']='datediff_firstoccur_int2'

        ylim1_dict={}
        ylim1_dict['allcausemortality']=ylim1_survival_allcausemortality
        ylim1_dict['algoMACE']=ylim1_survival_algoMACE
        ylim1_dict['firstoccurMACE']=ylim1_survival_firstoccurMACE

        ylim2_dict={}
        ylim2_dict['allcausemortality']=ylim2_survival_allcausemortality
        ylim2_dict['algoMACE']=ylim2_survival_algoMACE
        ylim2_dict['firstoccurMACE']=ylim2_survival_firstoccurMACE

        def survivalgraph(catname1,catname2,outcome,ylim1,ylim2):
            global txt
            global txt_men_40
            global txt_men_50
            global txt_men_60
            global txt_women_40
            global txt_women_50
            global txt_women_60
            global txt_bothsex_40
            global txt_bothsex_50
            global txt_bothsex_60
            global txt_men_allages
            global txt_women_allages
            datediff_temp=outcome_datediff_dict[outcome]
            event_temp=outcome_event_dict[outcome]
            score_temp=scorenamedict[catname1]

            df_final_temp=df_final[df_final[datediff_temp]<=3650][['age','sex',catname2,datediff_temp,event_temp,score_temp]]
            df_final_temp=df_final_temp[df_final_temp[datediff_temp]>=0]

            #df_final_temp2=df_final_temp[[event_temp,score_temp]]
            #print(len(df_final_temp2))
            #df_final_temp2=df_final_temp2.dropna()
            #fpr,tpr,thres=roc_curve(df_final_temp2[event_temp],df_final_temp2[score_temp])
            #roc_auc = round(auc(fpr, tpr),3)
            #print(len(df_final_temp2),np.sum(df_final_temp2[event_temp]),roc_auc)

            fpr,tpr,thres=roc_curve(df_final_temp[event_temp],df_final_temp[score_temp])
            roc_auc = round(auc(fpr, tpr),3)

            df_final_temp_men_40=df_final_temp[df_final_temp['age']>=40]
            df_final_temp_men_40=df_final_temp_men_40[df_final_temp_men_40['age']<50]
            df_final_temp_men_40=df_final_temp_men_40[df_final_temp_men_40['sex']==1]
            df_final_temp_men_40.reset_index(inplace=True,drop=True)

            df_final_temp_men_50=df_final_temp[df_final_temp['age']>=50]
            df_final_temp_men_50=df_final_temp_men_50[df_final_temp_men_50['age']<60]
            df_final_temp_men_50=df_final_temp_men_50[df_final_temp_men_50['sex']==1]
            df_final_temp_men_50.reset_index(inplace=True,drop=True)

            df_final_temp_men_60=df_final_temp[df_final_temp['age']>=60]
            df_final_temp_men_60=df_final_temp_men_60[df_final_temp_men_60['age']<71]
            df_final_temp_men_60=df_final_temp_men_60[df_final_temp_men_60['sex']==1]
            df_final_temp_men_60.reset_index(inplace=True,drop=True)

            df_final_temp_women_40=df_final_temp[df_final_temp['age']>=40]
            df_final_temp_women_40=df_final_temp_women_40[df_final_temp_women_40['age']<50]
            df_final_temp_women_40=df_final_temp_women_40[df_final_temp_women_40['sex']==0]
            df_final_temp_women_40.reset_index(inplace=True,drop=True)

            df_final_temp_women_50=df_final_temp[df_final_temp['age']>=50]
            df_final_temp_women_50=df_final_temp_women_50[df_final_temp_women_50['age']<60]
            df_final_temp_women_50=df_final_temp_women_50[df_final_temp_women_50['sex']==0]
            df_final_temp_women_50.reset_index(inplace=True,drop=True)

            df_final_temp_women_60=df_final_temp[df_final_temp['age']>=60]
            df_final_temp_women_60=df_final_temp_women_60[df_final_temp_women_60['age']<71]
            df_final_temp_women_60=df_final_temp_women_60[df_final_temp_women_60['sex']==0]
            df_final_temp_women_60.reset_index(inplace=True,drop=True)

            df_final_temp_bothsex_40=df_final_temp[df_final_temp['age']>=40]
            df_final_temp_bothsex_40=df_final_temp_bothsex_40[df_final_temp_bothsex_40['age']<50]
            df_final_temp_bothsex_40.reset_index(inplace=True,drop=True)

            df_final_temp_bothsex_50=df_final_temp[df_final_temp['age']>=50]
            df_final_temp_bothsex_50=df_final_temp_bothsex_50[df_final_temp_bothsex_50['age']<60]
            df_final_temp_bothsex_50.reset_index(inplace=True,drop=True)

            df_final_temp_bothsex_60=df_final_temp[df_final_temp['age']>=60]
            df_final_temp_bothsex_60=df_final_temp_bothsex_60[df_final_temp_bothsex_60['age']<71]
            df_final_temp_bothsex_60.reset_index(inplace=True,drop=True)

            df_final_temp_men_allages=df_final_temp[df_final_temp['sex']==1]
            df_final_temp_men_allages.reset_index(inplace=True,drop=True)

            df_final_temp_women_allages=df_final_temp[df_final_temp['sex']==0]
            df_final_temp_women_allages.reset_index(inplace=True,drop=True)

            fpr,tpr,thres=roc_curve(df_final_temp_men_40[event_temp],df_final_temp_men_40[score_temp])
            roc_auc_men_40 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_men_50[event_temp],df_final_temp_men_50[score_temp])
            roc_auc_men_50 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_men_60[event_temp],df_final_temp_men_60[score_temp])
            roc_auc_men_60 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_40[event_temp],df_final_temp_women_40[score_temp])
            roc_auc_women_40 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_50[event_temp],df_final_temp_women_50[score_temp])
            roc_auc_women_50 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_60[event_temp],df_final_temp_women_60[score_temp])
            roc_auc_women_60 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_bothsex_40[event_temp],df_final_temp_bothsex_40[score_temp])
            roc_auc_bothsex_40 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_bothsex_50[event_temp],df_final_temp_bothsex_50[score_temp])
            roc_auc_bothsex_50 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_bothsex_60[event_temp],df_final_temp_bothsex_60[score_temp])
            roc_auc_bothsex_60 = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_men_allages[event_temp],df_final_temp_men_allages[score_temp])
            roc_auc_men_allages = round(auc(fpr, tpr),3)

            fpr,tpr,thres=roc_curve(df_final_temp_women_allages[event_temp],df_final_temp_women_allages[score_temp])
            roc_auc_women_allages = round(auc(fpr, tpr),3)



            cat1 = df_final_temp[df_final_temp[catname2] == 1]
            cat2 = df_final_temp[df_final_temp[catname2] == 2]
            cat3 = df_final_temp[df_final_temp[catname2] == 3]
            print(outcome,catname2, len(cat1),len(cat2),len(cat3))

            kmf = KaplanMeierFitter()
            kmf.fit(cat1[datediff_temp], cat1[event_temp], label='Low')
            ax_kmf = kmf.plot(loc=slice(0.,3650.))
            kmf.fit(cat2[datediff_temp], cat2[event_temp], label='Moderate')
            ax_kmf = kmf.plot(ax = ax_kmf,loc=slice(0.,3650.))
            kmf.fit(cat3[datediff_temp], cat3[event_temp], label='High')
            ax_kmf = kmf.plot(ax = ax_kmf,loc=slice(0.,3650.))

            ax_kmf.set_ylim([ylim1, ylim2])
            ax_kmf.set_yticks(np.arange(ylim1, ylim2, step=0.02))
            ax_kmf.set_xlabel('Time (days)')
            ax_kmf.set_ylabel('Survival Rate')
            ['ChatGPT (Original)','ChatGPT (6:3:1)','Framingham (Original)','Framingham (6:3:1)' ,'ACC/AHA ASCVD (Original)','ACC/AHA ASCVD (6:3:1)']
            if catname1=='ChatGPT (Original)':
                ax_kmf.set_title('GPT-4')
            elif catname1=='Framingham (Original)':
                ax_kmf.set_title('Framingham')
            elif catname1=='ACC/AHA ASCVD (Original)':
                ax_kmf.set_title('ACC/AHA')
            else:
                ax_kmf.set_title(catname1)





            try:
                os.mkdir(newtablename)
            except:
                pass
            try:
                os.mkdir(newtablename+'/survival')
            except:
                pass
            catname1_temp=catname1.replace(' ','_')
            catname1_temp=catname1_temp.replace('(','_')
            catname1_temp=catname1_temp.replace(')','')
            catname1_temp=catname1_temp.replace(':','')
            catname1_temp=catname1_temp.replace('/','')
            ax_kmf.get_figure().savefig(newtablename+"/survival/"+outcome+'_' +catname1_temp+   ".png",transparent=True)
            plt.close('all')

            if catname1=='ChatGPT (Original)':
                txt+= ('ChatGPT' + ' AUROC = ' + str(roc_auc) +'\n' )
                txt_men_40+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_40) +'\n' )
                txt_men_50+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_50) +'\n' )
                txt_men_60+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_60) +'\n' )
                txt_women_40+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_40) +'\n' )
                txt_women_50+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_50) +'\n' )
                txt_women_60+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_60) +'\n' )
                txt_bothsex_40+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_bothsex_40) +'\n' )
                txt_bothsex_50+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_bothsex_50) +'\n' )
                txt_bothsex_60+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_bothsex_60) +'\n' )
                txt_men_allages+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_men_allages) +'\n' )
                txt_women_allages+= ('ChatGPT' + ' AUROC = ' + str(roc_auc_women_allages) +'\n' )
            elif catname1=='Framingham (Original)':
                txt+= ('Framingham' + ' AUROC = ' + str(roc_auc) +'\n' )
                txt_men_40+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_40) +'\n' )
                txt_men_50+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_50) +'\n' )
                txt_men_60+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_60) +'\n' )
                txt_women_40+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_40) +'\n' )
                txt_women_50+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_50) +'\n' )
                txt_women_60+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_60) +'\n' )
                txt_bothsex_40+= ('Framingham' + ' AUROC = ' + str(roc_auc_bothsex_40) +'\n' )
                txt_bothsex_50+= ('Framingham' + ' AUROC = ' + str(roc_auc_bothsex_50) +'\n' )
                txt_bothsex_60+= ('Framingham' + ' AUROC = ' + str(roc_auc_bothsex_60) +'\n' )
                txt_men_allages+= ('Framingham' + ' AUROC = ' + str(roc_auc_men_allages) +'\n' )
                txt_women_allages+= ('Framingham' + ' AUROC = ' + str(roc_auc_women_allages) +'\n' )
            elif catname1=='ACC/AHA ASCVD (Original)':
                txt+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc) +'\n' )
                txt_men_40+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_40) +'\n' )
                txt_men_50+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_50) +'\n' )
                txt_men_60+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_60) +'\n' )
                txt_women_40+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_40) +'\n' )
                txt_women_50+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_50) +'\n' )
                txt_women_60+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_60) +'\n' )
                txt_bothsex_40+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_bothsex_40) +'\n' )
                txt_bothsex_50+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_bothsex_50) +'\n' )
                txt_bothsex_60+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_bothsex_60) +'\n' )
                txt_men_allages+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_men_allages) +'\n' )
                txt_women_allages+= ('ACC/AHA' + ' AUROC = ' + str(roc_auc_women_allages) +'\n' )
            else:
                pass




        for outcome in ['allcausemortality','algoMACE','firstoccurMACE']:
            count=0
            for catname1 in ['ChatGPT (Original)','ChatGPT (6:3:1)','Framingham (Original)','Framingham (6:3:1)' ,'ACC/AHA ASCVD (Original)','ACC/AHA ASCVD (6:3:1)']:
                if count==0:
                    txt+='===== ' +outcome+' ====='+'\n'
                    txt_men_40+='===== ' +outcome+' ====='+'\n'
                    txt_men_50+='===== ' +outcome+' ====='+'\n'
                    txt_men_60+='===== ' +outcome+' ====='+'\n'
                    txt_women_40+='===== ' +outcome+' ====='+'\n'
                    txt_women_50+='===== ' +outcome+' ====='+'\n'
                    txt_women_60+='===== ' +outcome+' ====='+'\n'
                    txt_bothsex_40+='===== ' +outcome+' ====='+'\n'
                    txt_bothsex_50+='===== ' +outcome+' ====='+'\n'
                    txt_bothsex_60+='===== ' +outcome+' ====='+'\n'
                    txt_men_allages+='===== ' +outcome+' ====='+'\n'
                    txt_women_allages+='===== ' +outcome+' ====='+'\n'
                    count+=1

                ylim1_survival=ylim1_dict[outcome]
                ylim2_survival=ylim2_dict[outcome]
                catname2=catnamedict[catname1]
                survivalgraph(catname1,catname2,outcome,ylim1_survival,ylim2_survival)


        file = open(newtablename+"/AUROC.txt", "w") 
        file.write(txt)
        file.close()

        file = open(newtablename+"/AUROC_men_40.txt", "w") 
        file.write(txt_men_40)
        file.close()

        file = open(newtablename+"/AUROC_men_50.txt", "w") 
        file.write(txt_men_50)
        file.close()

        file = open(newtablename+"/AUROC_men_60.txt", "w") 
        file.write(txt_men_60)
        file.close()

        file = open(newtablename+"/AUROC_women_40.txt", "w") 
        file.write(txt_women_40)
        file.close()

        file = open(newtablename+"/AUROC_women_50.txt", "w") 
        file.write(txt_women_50)
        file.close()

        file = open(newtablename+"/AUROC_women_60.txt", "w") 
        file.write(txt_women_60)
        file.close()

        file = open(newtablename+"/AUROC_bothsex_40.txt", "w") 
        file.write(txt_bothsex_40)
        file.close()

        file = open(newtablename+"/AUROC_bothsex_50.txt", "w") 
        file.write(txt_bothsex_50)
        file.close()

        file = open(newtablename+"/AUROC_bothsex_60.txt", "w") 
        file.write(txt_bothsex_60)
        file.close()

        file = open(newtablename+"/AUROC_men_allages.txt", "w") 
        file.write(txt_men_allages)
        file.close()

        file = open(newtablename+"/AUROC_women_allages.txt", "w") 
        file.write(txt_women_allages)
        file.close()

        ######## 설정변수 ##########
        lim=80


        ############################


        txt=''

        def scatter_plot(xxx,yyy,lim):
            global txt
            plt.figure(figsize=(10,10))
            if xxx=='ChatGPT':
                xxxx='GPT-4'
            else:
                xxxx=xxx
            if yyy=='ACC/AHA':
                yyyy='ACC/AHA'
            else:
                yyyy=yyy



            plt.title(xxxx+ ' VS '+ yyyy , fontsize=20)



            if xxx=='ChatGPT':
                plt.xlabel('GPT-4', fontsize=14)
            else:
                plt.xlabel(xxx, fontsize=14)
            if yyy=='ACC/AHA':
                plt.ylabel('ACC/AHA', fontsize=14)
            else:
                plt.ylabel(yyy, fontsize=14)

            plt.xlim([-5, lim])
            plt.ylim([-5, lim])
            plt.yticks(np.arange(0, lim+10, step=10))
            plt.xticks(np.arange(0, lim+10, step=10))
            if xxx=='ChatGPT':
                xxx_column='score_final'
            elif xxx=='Framingham':
                xxx_column='Framingham'
            elif xxx=='ACC/AHA':
                xxx_column='ACC/AHA'
            else:
                print('error')

            if yyy=='ChatGPT':
                yyy_column='score_final'
            elif yyy=='Framingham':
                yyy_column='Framingham'
            elif yyy=='ACC/AHA':
                yyy_column='ACC/AHA'
            else:
                print('error')    

            try:
                os.mkdir(newtablename)
            except:
                pass
            try:
                os.mkdir(newtablename+'/scatter')
            except:
                pass

            xxx=xxx.replace('/','_')
            yyy=yyy.replace('/','_')

            plt.scatter(df_final[xxx_column],df_final[yyy_column],s=0.07, alpha=0.6)
            plt.savefig(newtablename+"/scatter/"+xxx+'_'+yyy+".png",transparent=True)
            plt.close('all')

            pearr=round(pearsonr(df_final[xxx_column],df_final[yyy_column]).statistic,3)
            pearp=round(pearsonr(df_final[xxx_column],df_final[yyy_column]).pvalue,3)

            txt+='==========================\n'
            txt+=('--- '+xxx+ ' VS '+ yyy + ' ---'+'\n')
            txt+=('pearson r = ' + str(pearr)+', pearson pvalue = '+str(pearp)   + '\n')
            txt+=(   xxx +'_mean = '+ str(round(np.mean(df_final[xxx_column]),3))   + '\n')
            txt+=(   yyy +'_mean = '+ str(round(np.mean(df_final[yyy_column]),3))   + '\n')
            txt+=(   xxx +'_shapiro_pvalue = '+ str(round(shapiro(df_final[xxx_column]).pvalue,3))   + '\n')
            txt+=(   yyy +'_shapiro_pvalue = '+ str(round(shapiro(df_final[yyy_column]).pvalue,3))   + '\n')
            txt+=(   'mannwhitneyu_pvalue = '+ str(round(mwu(df_final[xxx_column],df_final[yyy_column]).pvalue,3))   + '\n')



        for xxx,yyy in [['ChatGPT','Framingham'],['ChatGPT','ACC/AHA'],['Framingham','ACC/AHA']]:
            scatter_plot(xxx,yyy,lim)

        txt+='=========================='

        file = open(newtablename+"/scatter/"+'stats.txt', "w") 
        file.write(txt)
        file.close()



        #######################################################

        score_individuals=[]
        for sss in range(iterations):
            score_individuals.append('score'+str(sss))
        df_final_temp=df_final[df_final['datediff_algo_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_algo_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)
        Framingham_temp_algo=df_final_temp['Framingham']
        df_final_temp_algo_scores=df_final_temp[score_individuals]

        df_final_temp=df_final[df_final['datediff_firstoccur_int2']<=3650]
        df_final_temp=df_final_temp[df_final_temp['datediff_firstoccur_int2']>=0]
        df_final_temp.reset_index(inplace=True,drop=True)
        Framingham_temp_firstoccur=df_final_temp['Framingham']
        df_final_temp_firstoccur_scores=df_final_temp[score_individuals]

        mean=df_final_temp_firstoccur_scores.mean(axis=1)
        std=df_final_temp_firstoccur_scores.std(axis=1)
        df_final_temp_firstoccur_scores['mean']=mean
        df_final_temp_firstoccur_scores['std']=std
        df_final_temp_firstoccur_scores['CV']=df_final_temp_firstoccur_scores['std']/df_final_temp_firstoccur_scores['mean']

        df_final_temp_firstoccur_scores['upper']=df_final_temp_firstoccur_scores['mean']+df_final_temp_firstoccur_scores['std']
        df_final_temp_firstoccur_scores['under']=df_final_temp_firstoccur_scores['mean']-df_final_temp_firstoccur_scores['std']
        df_final_temp_firstoccur_scores['Framingham']=Framingham_temp_firstoccur

        df_final_temp_firstoccur_scores_low=df_final_temp_firstoccur_scores[df_final_temp_firstoccur_scores['mean']<=10].reset_index(drop=True)
        df_final_temp_firstoccur_scores_mod=df_final_temp_firstoccur_scores[df_final_temp_firstoccur_scores['mean']<=20].reset_index(drop=True)
        df_final_temp_firstoccur_scores_mod=df_final_temp_firstoccur_scores_mod[df_final_temp_firstoccur_scores_mod['mean']>10].reset_index(drop=True)
        df_final_temp_firstoccur_scores_hig=df_final_temp_firstoccur_scores[df_final_temp_firstoccur_scores['mean']>30].reset_index(drop=True)

        firstoccur_total_CV=np.mean(df_final_temp_firstoccur_scores['CV'])
        firstoccur_total_sd=np.mean(df_final_temp_firstoccur_scores['std'])
        firstoccur_low_CV=np.mean(df_final_temp_firstoccur_scores_low['CV'])
        firstoccur_mod_CV=np.mean(df_final_temp_firstoccur_scores_mod['CV'])
        firstoccur_hig_CV=np.mean(df_final_temp_firstoccur_scores_hig['CV'])
        firstoccur_low_sd=np.mean(df_final_temp_firstoccur_scores_low['std'])
        firstoccur_mod_sd=np.mean(df_final_temp_firstoccur_scores_mod['std'])
        firstoccur_hig_sd=np.mean(df_final_temp_firstoccur_scores_hig['std'])






        mean=df_final_temp_algo_scores.mean(axis=1)
        std=df_final_temp_algo_scores.std(axis=1)
        df_final_temp_algo_scores['mean']=mean
        df_final_temp_algo_scores['std']=std
        df_final_temp_algo_scores['CV']=df_final_temp_algo_scores['std']/df_final_temp_algo_scores['mean']

        df_final_temp_algo_scores['upper']=df_final_temp_algo_scores['mean']+df_final_temp_algo_scores['std']
        df_final_temp_algo_scores['under']=df_final_temp_algo_scores['mean']-df_final_temp_algo_scores['std']
        df_final_temp_algo_scores['Framingham']=Framingham_temp_algo

        df_final_temp_algo_scores_low=df_final_temp_algo_scores[df_final_temp_algo_scores['mean']<=10].reset_index(drop=True)
        df_final_temp_algo_scores_mod=df_final_temp_algo_scores[df_final_temp_algo_scores['mean']<=20].reset_index(drop=True)
        df_final_temp_algo_scores_mod=df_final_temp_algo_scores_mod[df_final_temp_algo_scores_mod['mean']>10].reset_index(drop=True)
        df_final_temp_algo_scores_hig=df_final_temp_algo_scores[df_final_temp_algo_scores['mean']>30].reset_index(drop=True)

        algo_total_CV=np.mean(df_final_temp_algo_scores['CV'])
        algo_total_sd=np.mean(df_final_temp_algo_scores['std'])
        algo_low_CV=np.mean(df_final_temp_algo_scores_low['CV'])
        algo_mod_CV=np.mean(df_final_temp_algo_scores_mod['CV'])
        algo_hig_CV=np.mean(df_final_temp_algo_scores_hig['CV'])
        algo_low_sd=np.mean(df_final_temp_algo_scores_low['std'])
        algo_mod_sd=np.mean(df_final_temp_algo_scores_mod['std'])
        algo_hig_sd=np.mean(df_final_temp_algo_scores_hig['std'])

        firstoccurcvsdlist=[firstoccur_total_CV,firstoccur_total_sd,firstoccur_low_CV,firstoccur_mod_CV,firstoccur_hig_CV,firstoccur_low_sd,firstoccur_mod_sd,firstoccur_hig_sd]
        algocvsdlist=[algo_total_CV,algo_total_sd,algo_low_CV,algo_mod_CV,algo_hig_CV,algo_low_sd,algo_mod_sd,algo_hig_sd]


        cvsddf=pd.DataFrame({'algo':algocvsdlist,'firstoccur':firstoccurcvsdlist})
        cvsddf.index=['total_CV','total_SD','low_CV','mod_CV','high_CV','low_SD','mod_SD','high_SD']
        cvsddf.to_csv(newtablename+'/CV_SD.csv')







        #######################################################




        dfdfdf2=df_final[['age','sex','total_chol','HDL','LDL','TG','Glucose','SBP','DBP','height','weight','Smoking','med_final','DM_final','Framingham','ACC/AHA','framingham_cat','ACC_AHA_cat','category_final','score_final']]
        dfdfdf2.columns=['Age','Sex','Total_chol','HDL','LDL','TG','Glucose','SBP','DBP','Height','Weight','Smoking','BP medication','DM','Framingham','ACC/AHA','Framingham category','ACC/AHA category','ChatGPT category','ChatGPT score']

        catcolumns=['Smoking','Sex','DM','BP medication','ChatGPT category','Framingham category','ACC/AHA category']

        def table1(catcolumn):
            nonnormallist=[]
            def _normality(self, x):
                #print(x.name)

                if len(x.values[~np.isnan(x.values)]) >= 20:
                    p = stats.shapiro(x.values).pvalue
                else:
                    p = None
                # dropna=False argument in pivot_table does not function as expected
                # return -1 instead of None
                if pd.isnull(p):
                    return -1
                if p<=0.05:
                    nonnormallist.append(x.name)
                return p

            TableOne._normality=_normality

            def my_custom_test(group1, group2):
                """
                Hypothesis test for test_self_defined_statistical_tests
                """
                my_custom_test.__name__ = "mannwhitneyu"
                _, pval= scipy.stats.mannwhitneyu(group1, group2)
                return pval

            nonnormallist=[]
            def _normality(self, x):
                #print(x.name)

                if len(x.values[~np.isnan(x.values)]) >= 20:
                    p = stats.shapiro(x.values).pvalue
                else:
                    p = None
                # dropna=False argument in pivot_table does not function as expected
                # return -1 instead of None
                if pd.isnull(p):
                    return -1
                if p<=0.05:
                    nonnormallist.append(x.name)
                return p

            TableOne._normality=_normality

            table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True)
            nonnormallist=list(set(nonnormallist))
            nonnormallist

            table1=TableOne(dfdfdf2,categorical=catcolumns,groupby=[catcolumn],normal_test=True,pval=True,htest_name=True,nonnormal=nonnormallist)
            try:
                os.mkdir(newtablename)
            except:
                pass
            try:
                os.mkdir(newtablename+'/table1')
            except:
                pass
            catcolumn=catcolumn.replace(' ','_')
            catcolumn=catcolumn.replace('/','_')
            table1.to_html(newtablename+'/table1/'+catcolumn+'.html')

        for catcolumn in ['ChatGPT category','Framingham category','ACC/AHA category']:
            table1(catcolumn)

# individual AUROCs

In [ ]:
df_final_temp=df_final[df_final['datediff_algo_int2']<=3650]
df_final_temp=df_final_temp[df_final_temp['datediff_algo_int2']>=0]
df_final_temp.reset_index(inplace=True,drop=True)
print(len(df_final_temp))
df_final_temp_algo=df_final_temp[['score0','score1','score2','score3','score4','algo_event']]
print(np.sum(df_final_temp_algo['algo_event']))

df_final_temp=df_final[df_final['datediff_firstoccur_int2']<=3650]
df_final_temp=df_final_temp[df_final_temp['datediff_firstoccur_int2']>=0]
df_final_temp.reset_index(inplace=True,drop=True)
Framingham_temp_firstoccur=df_final_temp['Framingham']
print(len(df_final_temp))
df_final_temp_firstoccur=df_final_temp[['score0','score1','score2','score3','score4','firstoccur_event']]
print(np.sum(df_final_temp_firstoccur['firstoccur_event']))

for dfdfdf in ['df_final_temp_algo','df_final_temp_firstoccur']:
    dfdfdf2=globals()[dfdfdf]
    for scoreint in ['score0','score1','score2','score3','score4']:
        fpr,tpr,thres=roc_curve(dfdfdf2[dfdfdf.split('_')[-1]+'_event'],dfdfdf2[scoreint])
        roc_auc_temp = round(auc(fpr, tpr),3)
        print(dfdfdf.split('_')[-1],scoreint,roc_auc_temp)